In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from mpl_toolkits.axes_grid1 import make_axes_locatable

sns.set_theme()
%config InlineBackend.figure_formats = ['svg']
%matplotlib qt

# Settings

In [13]:
M = 100

M_cutoff = 20

Bx = 1.
dB0 = 1.
n0 = 1.
v = 1.

t0 = 0.0001
dt = 0.0002
tmax = 0.01

sym_fix = True
random_init = False

b_t_in_non_lin = True

frame_time = 50

# Simulation Function Definitions

In [10]:
#############
# CONSTANTS #
#############

def B(t):
    global n0, v, dB0
    return dB0 - 2. * t * n0 + 3. * v * n0**2
def C(t):
    global n0
    return -1.* (t + 3. * n0)

#############
# FUNCTIONS #
#############

def g_sq_hat_sym(eta_i):

    global kx2, kx3, kx4
    global ky2, ky3, ky4
    global G

    ret = kx4 + ky4
    ret += 2. * np.linalg.norm(G[eta_i]) * ( kx3 + ky3 )
    ret += 4. * np.linalg.norm(G[eta_i])**2 * ( kx2 + ky2 )

    return ret

def g_sq_hat_non_sym(eta_i):

    global kx2, kx3, kx4
    global ky2, ky3, ky4
    global G

    ret = kx4 + ky4
    ret += 2. * ( G[eta_i][0] * kx3 + G[eta_i][1] * ky3 )
    ret += 4. * np.linalg.norm(G[eta_i])**2 * ( kx2 + ky2 )

    return ret

g_sq_hat = g_sq_hat_sym if sym_fix else g_sq_hat_non_sym

def n_hat_no_bt(eta_i, t):

    global etas, D

    poss_eta_is = {0, 1, 2}
    other_etas = list(poss_eta_is.difference({eta_i}))

    C_t = C(t)

    eta_abs = np.power(np.abs(etas[eta_i]), 2)
    eta_conj1 = np.conj(etas[other_etas[0]])
    eta_conj2 = np.conj(etas[other_etas[1]])

    n = 3. * D * eta_abs * etas[eta_i]
    n += 2. * C_t * eta_conj1 * eta_conj2
    n = np.fft.fft2(n)

    return -1. * n * np.linalg.norm(G[eta_i])**2

def n_hat_bt(eta_i, t):

    global etas, D

    poss_eta_is = {0, 1, 2}
    other_etas = list(poss_eta_is.difference({eta_i}))

    C_t = C(t)
    B_t = B(t)

    eta_abs = np.power(np.abs(etas[eta_i]), 2)
    eta_conj1 = np.conj(etas[other_etas[0]])
    eta_conj2 = np.conj(etas[other_etas[1]])

    n = 3. * D * eta_abs * etas[eta_i]
    n += 2. * C_t * eta_conj1 * eta_conj2
    n += B_t * etas[eta_i]
    n = np.fft.fft2(n)

    return -1. * n * np.linalg.norm(G[eta_i])**2

n_hat = n_hat_bt if b_t_in_non_lin else n_hat_no_bt

def lagr_hat_bt(eta_i, t):

    global A, etas, D, G

    B_t = B(t)
    G_op = g_sq_hat(eta_i)

    poss_eta_is = {0, 1, 2}
    other_etas = list(poss_eta_is.difference({eta_i}))

    eta_abs1 = np.power(np.abs(etas[other_etas[0]]), 2)
    eta_abs2 = np.power(np.abs(etas[other_etas[1]]), 2)

    Gm_sq = (np.linalg.norm(G[eta_i])**2).astype(complex)

    lagr = A * G_op
    lagr = lagr.astype(complex)
    lagr += B_t 
    lagr += 6. * D * np.fft.fft2(eta_abs1) 
    lagr += 6. * D * np.fft.fft2(eta_abs2)

    print(-1. * lagr)
    print(-1. * lagr * np.linalg.norm(G[eta_i])**2)

    return -1. * lagr * Gm_sq

def lagr_hat_no_bt(eta_i, t):

    global A, etas, D, G

    G_op = g_sq_hat(eta_i)

    poss_eta_is = {0, 1, 2}
    other_etas = list(poss_eta_is.difference({eta_i}))

    eta_abs1 = np.power(np.abs(etas[other_etas[0]]), 2)
    eta_abs2 = np.power(np.abs(etas[other_etas[1]]), 2)

    Gm_sq = (np.linalg.norm(G[eta_i])**2).astype(complex)

    lagr = A * G_op
    lagr = lagr.astype(complex)
    lagr += 6. * D * np.fft.fft2(eta_abs1) 
    lagr += 6. * D * np.fft.fft2(eta_abs2)

    return -1. * lagr * Gm_sq

lagr_hat = lagr_hat_no_bt if b_t_in_non_lin else lagr_hat_bt

def eta_routine(eta_i, t, dt):

    lagr = lagr_hat(eta_i, t)
    n = n_hat(eta_i, t)

    exp_lagr = np.exp(lagr * dt)

    n_eta = exp_lagr * np.fft.fft2(etas[eta_i])
    n_eta += (exp_lagr - 1.) / lagr * n 

    return np.fft.ifft2(n_eta)

# Plotting Functions / Simulation Run

In [4]:
def plot(eta_i):

    plt.ioff()

    global etas, xm, ym

    eta_sum = np.abs(etas[0]) + np.abs(etas[1]) + np.abs(etas[2])
    plt.figure(figsize=(13, 10))
    plt.contourf(xm, ym, eta_sum)
    plt.grid("on")
    plt.colorbar()
    plt.savefig(f"/home/max/projects/apfc/data/{eta_i}.svg", format="svg")

def update(frame):
    
    global t, dt, etas, xm, ym, fig, ax, i

    n_eta1 = eta_routine(0, t, dt)
    n_eta2 = eta_routine(1, t, dt)
    n_eta3 = eta_routine(2, t, dt)

    etas[0,:,:] = n_eta1
    etas[1,:,:] = n_eta2
    etas[2,:,:] = n_eta3

    eta_sum = np.abs(etas[0]) + np.abs(etas[1]) + np.abs(etas[2])

    ax.clear()

    cont = ax.contourf(xm, ym, eta_sum)
    ax.grid('on')
    plt.title(f"t = {t:.4f}, i = {i}, dt={dt:.4f}")

    if (t == t0):
        fig.colorbar(cont)

    t += dt
    i += 1

def plot_window(frame):

    global t, dt, etas, xm, ym, fig, ax, i, cax
    global Bx, dB0, n0, v

    n_eta1 = eta_routine(0, t, dt)
    n_eta2 = eta_routine(1, t, dt)
    n_eta3 = eta_routine(2, t, dt)

    etas[0,:,:] = n_eta1
    etas[1,:,:] = n_eta2
    etas[2,:,:] = n_eta3

    eta_sum = np.abs(etas[0]) + np.abs(etas[1]) + np.abs(etas[2])

    ax.cla()

    cont = ax.contourf(xm, ym, eta_sum)
    ax.grid('on')
    ax.set_title(f"t = {t:.4f}, i = {i}, dt={dt:.4f}\n$B^x={Bx:.4f}, \Delta B^0={dB0:.4f}, n_0={n0:.4f}, v={v:.4f}$")
    fig.colorbar(cont, cax=cax)
    plt.tight_layout()

    t += dt
    i += 1

# Init Simulation

In [14]:
N = 10 * M

A = Bx
D = v

G = [
    np.array([- np.sqrt(3.) / 2., -0.5]),
    np.array([0., 1.]),
    np.array([np.sqrt(3.) / 2., -0.5]),
]

###############
# REAL COORDS #
###############

x = np.array([i for i in range(-M, M + 1)])
y = x

xm, ym = np.meshgrid(x, y)

###############
# FREQ COORDS #
###############

kx = np.pi / np.max(x) * x
ky = kx # assuming even spacing

kxm, kym = np.meshgrid(kx, ky)

kx2 = np.power(kxm, 2)
kx3 = np.power(kxm, 3)
kx4 = np.power(kxm, 4)

ky2 = np.power(kym, 2)
ky3 = np.power(kym, 3)
ky4 = np.power(kym, 4)

#############
# INIT ETAS #
#############

cut = xm.shape[0] // 2

def generate_eta():
    return np.array([
        [
            complex(np.random.uniform(), np.random.uniform()) 
            if (cut - i)**2 + (cut - j)**2 <= M_cutoff**2 else complex(0., 0.)
            for j in range(xm.shape[0])
        ] for i in range(xm.shape[1])
    ])

def norm_dist(offset, sigma):

    global xm, ym

    prefac = 1. / (sigma * 2. * np.pi)
    expon = -1 * ( (xm - offset[0])**2 + (ym - offset[1])**2) / (2. * sigma**2)

    return prefac * np.exp(expon)

etas = np.zeros((3, xm.shape[0], xm.shape[1]), dtype=complex)

if (random_init):
    etas[0,:,:] = generate_eta()
    etas[1,:,:] = generate_eta()
    etas[2,:,:] = generate_eta()
else: 
    etas[0,:,:] = norm_dist(20. * G[0], 10.)
    etas[1,:,:] = norm_dist(20. * G[1], 10.)
    etas[2,:,:] = norm_dist(20. * G[2], 10.)

# Run routine

In [15]:
nstep = int(round(tmax / dt))

t = t0
i = 1

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect("equal")

div = make_axes_locatable(ax)
cax = div.append_axes("right", "5%", "5%")

#ani = FuncAnimation(fig, update, interval=1000, frames=100)
ani = FuncAnimation(plt.gcf(), plot_window, interval=frame_time, frames=100)
plt.show()

# Alternative Func Defs

In [ ]:
def g_sq_hat2(eta_i):

    global kx2, kx3, kx4
    global ky2, ky3, ky4
    global G

    lapl = kx2 + ky2
    nabl = 2. * (G[eta_i][0] * kx + G[eta_i][1] * ky)

    return lapl**2 + 2. * nabl * lapl + nabl**2

def n_hat2(eta_i, t):

    global etas, D

    poss_eta_is = {0, 1, 2}
    other_etas = list(poss_eta_is.difference({eta_i}))

    C_t = C(t)

    poss_eta_is = {0, 1, 2}
    other_etas = list(poss_eta_is.difference({eta_i}))

    eta_abs = np.power(np.abs(etas[eta_i]), 2)
    eta_abs1 = np.power(np.abs(etas[other_etas[0]]), 2)
    eta_abs2 = np.power(np.abs(etas[other_etas[1]]), 2)

    eta_conj1 = np.conj(etas[other_etas[0]])
    eta_conj2 = np.conj(etas[other_etas[1]])

    n = 3. * D * eta_abs * etas[eta_i]
    n += 2. * C_t * eta_conj1 * eta_conj2
    n += 6. * D * eta_abs1 * etas[eta_i]
    n += 6. * D * eta_abs2 * etas[eta_i]

    n = np.fft.fft2(n)

    return -1. * n

def lagr_hat2(eta_i, t):

    global A, etas, D

    B_t = B(t)
    G = g_sq_hat(eta_i)

    lagr = A * G 
    lagr = lagr.astype(complex)
    lagr += B_t 

    return -1. * lagr

In [ ]:
np.linalg.norm(G[2])**2